# Theis pumping test - Fitting Theis to measured data
This notebook demonstrate the application of the Theis principle for pumping test evaluation in confined, transient setups. The notebook is based on an Spreadsheet from Prof. Rudolf Liedl.

## Introduction

### General situation
We consider a confined aquifer with constant transmissivity. If a well is pumping water out of the aquifer, radial flow towards the well is induced. The following figure illustrates this situation.

<img src="FIGS/Flow_well_confined.png" width="400">

*Fig: Radial flow in a confined aquifer (Fetter, 1994)*

The calculate the hydraulic situation, the following simplified flow equation can be used. This equation accounts for 1D radial transient flow towards a fully penetrating well within a confined aquifer without further sinks and sources:

$$ \frac{\partial^2 h}{\partial r^2}+\frac{1}{r}\frac{\partial h}{\partial r}=\frac{S}{T}\frac{\partial h}{\partial t} $$

### Solution by Theis for confined aquifers
Charles V. Theis presented a solution for this by deriving

$$ s(r,t)=\frac{Q}{4\pi T}W(u) $$

with the well function

$$ W(u) = \int_{u }^{+\infty} \frac{e^{-\tilde u}}{\tilde u}d\tilde u $$

and the dimensionless variable 

$$ u = \frac{Sr^2}{4Tt} $$

This equations are not easy to solve. Historically, values for the well function were provided by tables or as so called type-curve. The type-curve matching with experimental data for pumping test analysis can be considered as one of the basic hydrogeological methods.

However, modern computer provide an easier and more convinient way to solve the 1D radial flow equation based on the Theis approach. Subsequently, the Theis equation is solved with Python routines. The results for the measured data are graphically presented.

#### Curve fitting with s-t plots

With this approach, time and drawdown are provided by the Theis solution with 

$$ s =\frac{Q}{4\pi T}W(u) $$

and 

$$ t = \frac{Sr^2}{4T} \frac{1}{u} $$

Subsequently, time and drawdown are plotted and fitted to measured time and drawdown data.

### Solution by Neuman for unconfined aquifers

_PLEASE ADD SOME EXPLANATION HERE_

In [1]:
# (First, the necessary Python functions are initiated.)

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.special
from ipywidgets import *
#from scipy.special import expn # musste ich zufuegen-> warum ist mior nicht ganz klar da das Paket vorher ja schon geladen wird! Kann wieder raus; sollte  Über scipy.special.expn funktionieren

**The following section stores the measured data.**

This are two lists with time (in minutes) and drawdown (in meters). Make sure that those elements match each other (e.g. the first time is associated with the first drawdown value etc.)

**Subsequently, the given data for the processing of measured data are provided.**

This are the given data like
- pumping rate $Q$,
- aquifer thickness $b$,
- distance of the observation (where time and drawdown were measured) to the pumping well $r$.

**These data can be adjusted for individual measurements.**

In [33]:
# m_time = [0.50,1.00,1.33,2.00,3.00,5.00,7.00,9.33,13.50,22.00,30.00] # time in minutes
# m_ddown = [0.30,0.37,0.38,0.40,0.43,0.49,0.52,0.53,0.55,0.56,0.57]   # drawdown in meters

# Data from SYMPLE exercise
m_time = [1,1.5,2,2.5,3,4,5,6,8,10,12,14,18,24,30,40,50,60,100,120] # time in minutes
m_time_s = [i*60 for i in m_time] # time in seconds
m_ddown = [0.66,0.87,0.99,1.11,1.21,1.36,1.49,1.59,1.75,1.86,1.97,2.08,2.20,2.36,2.49,2.65,2.78,2.88,3.16,3.28]   # drawdown in meters
num_times = len(m_time)

# Parameters needed to solve Theis (From the SYMPLE example/excercise)
r = 120       # m
b = 8.5       # m
Qs = 0.3/60   # m^3/s
Qd = Qs*60*24 # m^3/d
#SyS = 0.1 # habe ich hier eingefuegt! Kann raus - wird über Slider eingelesen

# (Here the necessary functions like the well function $W(u)$ are defined. Later, those functions are used in the computation)
# Define a function, class, and object for Theis Well analysis

def well_function(u):
    return scipy.special.exp1(u)
    
# (Here, the methode computes the data for the well function. Those data can be used to generate a type curve.)
u_min = -5
u_max = 4

u = np.logspace(u_min,u_max)
u_inv = 1/u
w_u = well_function(u)

t = u_inv * t_term
s = w_u * s_term

# This is the function to plot the graph with the data, it is called by interact, arguments need to be provided by sliders etc.
def Theis_plot(T,b,Ss):
    # Compute K and SS to provide parameters for plausability check
    # (i.e. are the parameter in a reasonable range)
    K = T/b     # m/s
    S = Ss * b

    t_term = r**2 * S / 4 / T
    s_term = Qs/(4 * np.pi * T)

    t = u_inv * t_term
    s = w_u * s_term
        
    fig = plt.figure(figsize=(10,7))
    ax = fig.add_subplot(1, 1, 1)
    ax.plot(t, s, label=r'Computed drawdown')
    ax.plot(m_time_s, m_ddown,'ro', label=r'measured drawdown')
    plt.yscale("log")
    plt.xscale("log")
    plt.axis([1E-1,1E5,1E-4,1E+1])
    ax.set(xlabel='t', ylabel='s',title='Theis drawdown')
    ax.grid(which="both")
    plt.legend()
    plt.show()

    print("Transmissivity       T = ","% 10.2E"% T, " m^2/s")
    print("Storativity          S = ","% 10.2E"% S, "[-]")

**The pumping test evaluation is more or less the calibration of a model (i.e. the Theis equations) by adjusting parameters. Those parameters include**
- **Transmissivity** of the entire aquifer $T$
- **Storativity** of the aquifer $S$.

See what happens and modify the parameters $T$ and $S$. Modify until the fit is fine.

_(Hint: After selecting the slider you can use the mouse, your fingers, but also the arrow-keys of your keyboard to modify the parameter values.)_

In [34]:
interact(Theis_plot,
         T=widgets.FloatLogSlider(value=0.001,base=10,min=-6, max=3, step=0.1,readout=True,readout_format='.2e'),
         b = widgets.FloatSlider(value=20,min=2, max=200, step=0.1,readout=True,readout_format='.2f'),
         Ss = widgets.FloatLogSlider(value=0.0001,base=10,min=-9, max=1, step=0.1,readout=True,readout_format='.2e'))

interactive(children=(FloatLogSlider(value=0.001, description='T', max=3.0, min=-6.0, readout_format='.2e'), F…

<function __main__.Theis_plot(T, b, Ss)>